In [1]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch import nn
from resnet3d import generate_model
from timm.utils import AverageMeter

from tqdm import tqdm
import sys
import glob
import gc

# Parameters

In [2]:
BASE_PATH = '/home/junseonglee/01_codes/input/rsna-2023-abdominal-trauma-detection'
RESOL = 256
BATCH_SIZE = 2
LR = 0.0001
N_EPOCHS = 30
train_df = pd.read_csv(f'{BASE_PATH}/train.csv')
train_meta_df = pd.read_csv(f'{BASE_PATH}/train_meta.csv')
train_df = train_df.sort_values(by=['patient_id'])

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset

In [3]:
class AbdominalCTDataset(Dataset):
    def __init__(self, meta_df):
        self.meta_df = meta_df
    
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, idx):
        row = self.meta_df.iloc[idx]
        label = row[['bowel_healthy','bowel_injury',
                    'extravasation_healthy','extravasation_injury',
                    'kidney_healthy','kidney_low','kidney_high',
                    'liver_healthy','liver_low','liver_high',
                    'spleen_healthy','spleen_low','spleen_high',
                    'any_injury']]
        data_3d = cv2.imread(row['path'], cv2.IMREAD_GRAYSCALE).reshape(1, RESOL, RESOL, RESOL) # channel, 3D
        data_3d = torch.from_numpy(data_3d.astype(np.float32))
        label = label.to_numpy().astype(np.float32)
        label   = torch.from_numpy(label)
        return data_3d, label        

train_dataset = AbdominalCTDataset(train_meta_df)
#data_3d, label = train[0]
len(train_dataset)

del train_dataset
gc.collect()

16

# Model

In [4]:
class AbdominalClassifier(nn.Module):
    def __init__(self, model_depth, device = DEVICE):
        super().__init__()
        self.device = device
        self.resnet3d = generate_model(model_depth = model_depth, n_input_channels = 1)
        self.flatten  = nn.Flatten()
        self.softmax  = nn.Softmax(dim=1)
        self.sigmoid  = nn.Sigmoid()
        self.fc_bowel = nn.Linear(920960, 1)
        self.fc_extrav= nn.Linear(920960, 1)
        self.fc_kidney= nn.Linear(920960, 3)
        self.fc_liver = nn.Linear(920960, 3)
        self.fc_spleen= nn.Linear(920960, 3)

    def forward(self, x):
        x = self.resnet3d(x)
        for i in range(0, 4):
            x[i] = self.flatten(x[i])
        x = torch.cat(x, axis = 1)

        bowel = self.fc_bowel(x)
        bowel = self.sigmoid(bowel)        

        extrav= self.fc_extrav(x)
        extrav= self.sigmoid(extrav)

        kidney= self.fc_kidney(x)
        kidney= self.softmax(kidney)

        liver = self.fc_liver(x)
        liver = self.softmax(liver)

        spleen= self.fc_spleen(x)
        spleen= self.softmax(spleen)
        
        any_in= torch.cat([1-bowel, 1-extrav, kidney[:,:1], liver[:,:1], spleen[:,:1]], axis = 1)
        any_in= torch.ones(any_in.shape[0], 5).to(self.device) - any_in
        any_in= torch.max(any_in, dim = 1, keepdim= True).values
        labels = torch.cat([bowel, extrav, kidney, liver, spleen, any_in], dim = 1)
        #return bowel, extrav, kidney, liver, spleen, any_in
        return labels

In [5]:
model = AbdominalClassifier(18)

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(model))
del model
gc.collect()

43290571


0

In [6]:
train_dataset = AbdominalCTDataset(train_meta_df[train_meta_df['fold']!=0])
valid_dataset = AbdominalCTDataset(train_meta_df[train_meta_df['fold']==0])

train_loader = DataLoader(dataset = train_dataset, shuffle = True, batch_size = BATCH_SIZE, pin_memory = True, 
                          num_workers = 8, drop_last = False)

valid_loader = DataLoader(dataset = valid_dataset, shuffle = False, batch_size = BATCH_SIZE, pin_memory = True, 
                          num_workers = 8, drop_last = False)                        

# Train

In [7]:
model = AbdominalClassifier(18)
model.to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr = LR)
ttl_iters = N_EPOCHS * len(train_loader)
scheduler = CosineAnnealingLR(optimizer, T_max=ttl_iters, eta_min=1e-6)


weights = np.zeros(14)
weights[:] = 1, 2, 1, 6 ,1, 2, 4, 1, 2, 4, 1, 2, 4, 6

weights = np.ones((BATCH_SIZE, 1))
weights[:,0] = 2
crit_bowel  = nn.BCELoss(weight = torch.from_numpy(weights).to(DEVICE))
weights[:,0] = 6
crit_extrav = nn.BCELoss(weight = torch.from_numpy(weights).to(DEVICE))
crit_any_in = nn.BCELoss(weight = torch.from_numpy(weights).to(DEVICE))
weights = np.ones((3))
weights[1] = 2
weights[2] = 4
crit_kidney = nn.CrossEntropyLoss(weight = torch.from_numpy(weights).to(DEVICE), label_smoothing = 0.05)
crit_liver  = nn.CrossEntropyLoss(weight = torch.from_numpy(weights).to(DEVICE), label_smoothing = 0.05)
crit_spleen = nn.CrossEntropyLoss(weight = torch.from_numpy(weights).to(DEVICE), label_smoothing = 0.05)

In [8]:
for epoch in range(0, N_EPOCHS):
    train_meters = {'loss': AverageMeter()}
    model.train()
    for X, y in tqdm(train_loader, leave=False):
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()

        X_out = model(X)
        X_out = torch.log(X_out)
        print(X_out)
        print(y)
        loss  = crit_bowel(X_out[:,:1], y[:,:1])
        loss += crit_extrav(X_out[:,1:2], y[:,1:2])
        loss += crit_kidney(X_out[:,2:5], y[:,2:5])
        loss += cirt_liver(X_out[:,5:8], y[:,5:8])
        loss += crit_spleen(X_out[:,8:11], y[:,8:11])
        loss += crit_any_in(X_out[:,11:12], y[:,11:12])

        loss.backward()

        optimizer.step()
        scheduler.step()

        trn_loss = loss.item()      
        train_meters['loss'].update(trn_loss, n=X.size(0))          



  0%|          | 0/1891 [00:00<?, ?it/s]/home/junseonglee/miniconda3/envs/rapids-23.06/lib/python3.10/site-packages/torch/nn/modules/conv.py:608: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv3d(
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [0,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.


tensor([[-0.3924, -0.2410, -0.9311, -1.5638, -0.9249, -1.0646, -0.7324, -1.7466,
         -0.5842, -1.8121, -1.2761, -0.2410],
        [-0.2541, -0.2903, -0.5257, -1.3457, -1.9072, -0.6847, -1.0905, -1.8342,
         -0.6286, -2.2381, -1.0217, -0.2541]], device='cuda:0',
       grad_fn=<LogBackward0>)
tensor([[1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]],
       device='cuda:0')


RuntimeError: cross_entropy: weight tensor should be defined either for all 3 classes or no classes but got weight tensor of shape: [2, 3]